In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook as tqdm
%load_ext autoreload
%autoreload 2

In [2]:
!jupyter nbextension enable --py widgetsnbextension

/bin/bash: /home/rootroot/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

/bin/bash: /home/rootroot/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-03-30 10:27:31.352842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 10:27:32.581018: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/rootroot/miniconda3/envs/tf/lib/
2023-03-30 10:27:32.583265: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file 

In [4]:
import tensorflow as tf

2023-03-30 10:27:36.870362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 10:27:37.464994: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/rootroot/miniconda3/envs/tf/lib/
2023-03-30 10:27:37.466226: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/rootroot/miniconda3/envs/tf/lib/
2023-03-30 10:27:37.466234: W tensorflow/compiler/

In [5]:
import tensorflow_datasets as tfds

In [6]:
builder = tfds.builder('scientific_papers/arxiv')

In [7]:
ds = tfds.load('scientific_papers/arxiv', split='train[:75]', data_dir="../../data/")

2023-03-30 10:27:43.317042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 10:27:43.327759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 10:27:43.327813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 10:27:43.328204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

In [17]:
ds, info = tfds.load('scientific_papers/arxiv', data_dir="../../data/", shuffle_files=True, with_info = True)

In [18]:
train_dataset, test_dataset = ds['train'], ds['test']

In [19]:
train_dataset.element_spec

{'abstract': TensorSpec(shape=(), dtype=tf.string, name=None),
 'article': TensorSpec(shape=(), dtype=tf.string, name=None),
 'section_names': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [20]:
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)


In [25]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [26]:
df = tfds.as_dataframe(train_dataset.shuffle(BUFFER_SIZE).take(BUFFER_SIZE), info)

In [27]:
df.head(2)

,abstract,article,section_names
0,b' we study possible connection of the d+@xmat...,"b""it was demonstrated @xcite , that dp elastic...",b'introduction\nminimal relativization of the ...
1,b' in order to satisfy processing time constra...,b'music summarization has been the subject of ...,b'introduction\nmusic summarization\ngeneric s...


In [28]:
df.loc[0]

abstract         b' we study possible connection of the d+@xmat...
article          b"it was demonstrated @xcite , that dp elastic...
section_names    b'introduction\nminimal relativization of the ...
Name: 0, dtype: object

In [29]:
df.to_csv("../../data/sample_10k_data_for_visualization.csv", index = False)

In [41]:
for example, label in train_dataset.take(1):
    print('text: ', example.numpy())
    print("\n\n\n\n")
    print('label: ', label.numpy())

text:  b"it would be remarkable if the naive assumption that light linearly traces mass is correct .\nindeed , on scales of tens of kiloparsecs around galaxies it is known from rotation curve information that dark matter is more dispersed than starlight .\nit has long been entertained that dark matter might be more dispersed than light on megaparsec scales @xcite .\nif so , we would say there is a ` bias ' between what we see and the physically more fundamental parameter , the mass .\nsemi - analytic models @xcite anticipate variations in the conversion of baryons into stars and the manifestation as starlight at both very low and very high densities .\ntheoretical considerations prepare us for the possibility of a complex relationship between the distribution of mass and of the lighthouses sitting atop that mass . in this article\nwe present observational evidence for such complexity .\nthree environmental regimes will be considered . at home in the local group\nwe find ourselves in an

In [43]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [44]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'the formation of s0 galaxies is generally discussed in the framework of mergers of disk galaxies within a lambda cold dark matter ( @xmath4cdm ) cosmology . alternatively , they can be viewed as former spirals where star formation has ceased and gas lost by stripping mechanisms . however , less attention has been paid to the role of bars in the evolution of these galaxies .\nbars are known to be efficient drivers of gas towards the central regions of galaxies , and in the presence of nuclear bars @xcite or nuclear spirals , a central starburst might occur .\nnumerical simulations @xcite also predict that bars evolve due to angular momentum transfer between the bar and a massive or centrally concentrated halo , leading to more prominent bars .\nindeed , there is observational evidence that bars in early - type disk galaxies might be more evolved than bars in spiral galaxies .\nthey are longer @xcite , more massive , and have more frequently double - peaked fourier amplitude p

In [48]:
df

AttributeError: HTML formatting of column abstract failed:
 * feature: Text(shape=(), dtype=string)
 * input: array([b' vlt spectra of 14 luminous compact galaxies ( lcgs ) reveal strong metallic absorption line sytems as well as narrow and intense emission lines . \n their gas extinction is found to be large ( a@xmath0 @xmath1 1.5 mag ) leading to an upward revision of their star formation rate ( sfr ) to an average value of @xmath1 40 @xmath2 . \n large extinction values are also supported by the large rate of detection in one field observed by iso . \n gas metal abundances in lcgs have about half the solar value . \n lcg absorption spectra can be synthesized with a mix of a few gyr old and relatively metal - rich ( generally solar to over - solar values ) stellar population and a younger stellar population ( @xmath3 5@xmath4 years ) having a metal abundance similar to that of the gas . \n + we argue that lcgs are the progenitors of present - day spiral bulges . \n lcgs have masses a

                                            abstract  \
0  [b' vlt spectra of 14 luminous compact galaxie...   
1  [b' we continue the study of the question of w...   
2  [b" we report on the serendipitous discovery o...   
3  [b' condensed matter physicists have long soug...   
4  [b' standing spin wave mode frequencies and li...   
5  [b' we describe the interaction between an ele...   
6  [b" the major assumption of the lorentz  loren...   
7  [b' using data from recent laser - shock exper...   
8  [b' the energy spectra of galactic cosmic rays...   
9  [b' we analyze the perturbative and parametric...   

                                             article  
0  [b'the evolution of uv and ir luminosity densi...  
1  [b"recently , the question of when a pseudo - ...  
2  [b"strongly lensed galaxies are particularly u...  
3  [b'a spin liquid ( sl ) is a magnetic system w...  
4  [b'understanding the high - frequency dynamics...  
5  [b"the miniaturization of electronic devices a... 